In [10]:
from whoosh.fields import Schema, TEXT, DATETIME, ID

schema = Schema(title=TEXT(stored=True), post_content=TEXT(stored=True), parent_content=TEXT(stored=True), date=DATETIME(stored=True,sortable=True), author=ID(stored=True), link=ID(stored=True), category=ID(stored=True))

In [11]:
import os.path
from whoosh.index import create_in

if not os.path.exists("index"):
    os.mkdir("index")
ix = create_in("index", schema)

In [12]:
from whoosh.index import open_dir

ix = open_dir("index")

In [13]:
import json
with open("redtracker_full.json","r") as f:
    data = json.load(f)

In [14]:
writer = ix.writer()

In [16]:
from datetime import datetime
from dateutil.parser import parse

In [17]:
for d in data:
    d["date"] = parse(d["date"])
    writer.add_document(**d)

In [18]:
writer.commit()

In [15]:
from whoosh.qparser import QueryParser
parser = QueryParser("post_content", ix.schema)
myquery = parser.parse("Juggernauts")
with ix.searcher() as searcher:
    results = searcher.search(myquery, limit=50)
    for i in results:
        print(i)

In [71]:
searched_term = "achievement"

In [72]:
from whoosh.query import *
from whoosh.qparser import QueryParser
myquery = Term("post_content", searched_term)
#myquery = And([Term("post_content", "huge"), Term("post_content", "achievement")])

In [73]:
with ix.searcher() as searcher:
    results = searcher.search(myquery)
    print(results[0])
    for i in results:
        print(i)

<Hit {}>
<Hit {}>
<Hit {}>
<Hit {}>
<Hit {}>
<Hit {}>
<Hit {}>
<Hit {}>
<Hit {}>
<Hit {}>
<Hit {}>


In [63]:
results

<Top 4 Results for And([Term('post_content', 'huge'), Term('post_content', 'achievement')]) runtime=0.004702713340520859>

In [116]:
import os.path
from whoosh.index import create_in

if not os.path.exists("index_2"):
    os.mkdir("index_2")
    
schema = Schema(title=TEXT(stored=True), post_content=TEXT(stored=True), parent_content=TEXT(stored=True), date=TEXT(stored=True), author=TEXT(stored=True), link=TEXT(stored=True), category=TEXT(stored=True))
ix = create_in("index_2", schema)

In [117]:
ix = open_dir("index_2")

In [118]:
writer = ix.writer()

In [119]:
import json
with open("redtracker_full.json","r") as f:
    data = json.load(f)

In [120]:
for d in data[:20]:
    #d["date"] = parse(d["date"])
    writer.add_document(**d)

In [121]:
writer.commit()

In [142]:
searched_term = "Juggernauts"
myquery = Term("post_content", searched_term)
with ix.searcher() as searcher:
    results = searcher.search(myquery)
    for i in results:
        print(i)

In [16]:
results[0]

ReaderClosed: 

In [18]:
searcher = ix.searcher()

In [21]:

results = list(searcher.search(myquery))
for i in results:
    print(i)

<Hit {'author': 'Statikk', 'category': 'Champions & Gameplay Feedback - Now Disabled', 'date': datetime.datetime(2015, 8, 5, 2, 50, 7, 69000, tzinfo=tzlocal()), 'link': 'https://boards.pbe.leagueoflegends.com/en/c/champions-gameplay-feedback/Grx91k8I-darius-pbe-changelist-and-feedback?comment=000b00020000', 'parent_content': '<p>So, Statikk, played my first PBE match with Darius and have to say he felt in such a brief testing way more fair. Such a powerful champ that was able to frontload so much damage in lane always felt super hokey. Now with the delay on his Q, it feels (even when I\'m playing him) like there\'s some skill involved. And, the enemy has some indication and time to react. Felt good.</p>\n\n<p>He feels way slower than normal and it\'s definitely noticed, especially on an already slow champ. Could be that Trinity Force is a rush item on him (it\'s frequently built live anyways) for the Movespeed alone. Without I was having a heckuva time catching up to targets. Had to wa

In [22]:
res = results[0]

In [26]:
dict(res)

{'author': 'Statikk',
 'category': 'Champions & Gameplay Feedback - Now Disabled',
 'date': datetime.datetime(2015, 8, 5, 2, 50, 7, 69000, tzinfo=tzlocal()),
 'link': 'https://boards.pbe.leagueoflegends.com/en/c/champions-gameplay-feedback/Grx91k8I-darius-pbe-changelist-and-feedback?comment=000b00020000',
 'parent_content': '<p>So, Statikk, played my first PBE match with Darius and have to say he felt in such a brief testing way more fair. Such a powerful champ that was able to frontload so much damage in lane always felt super hokey. Now with the delay on his Q, it feels (even when I\'m playing him) like there\'s some skill involved. And, the enemy has some indication and time to react. Felt good.</p>\n\n<p>He feels way slower than normal and it\'s definitely noticed, especially on an already slow champ. Could be that Trinity Force is a rush item on him (it\'s frequently built live anyways) for the Movespeed alone. Without I was having a heckuva time catching up to targets. Had to wad

In [27]:
searcher.close()

In [39]:
from whoosh.qparser import FuzzyTermPlugin

In [54]:
parser = QueryParser("post_content", ix.schema)
parser.add_plugin(FuzzyTermPlugin())
myquery = parser.parse("Juggernauts")
with ix.searcher() as searcher:
    results = searcher.search(myquery, limit=50)
    print(results.estimated_length())
    results_list = []
    for i in results:
        results_list.append((dict(i), i.score))

188


In [55]:
results_list

[({'author': 'Statikk',
   'category': 'Champions & Gameplay Feedback - Now Disabled',
   'date': datetime.datetime(2015, 8, 5, 2, 50, 7, 69000, tzinfo=tzlocal()),
   'link': 'https://boards.pbe.leagueoflegends.com/en/c/champions-gameplay-feedback/Grx91k8I-darius-pbe-changelist-and-feedback?comment=000b00020000',
   'parent_content': '<p>So, Statikk, played my first PBE match with Darius and have to say he felt in such a brief testing way more fair. Such a powerful champ that was able to frontload so much damage in lane always felt super hokey. Now with the delay on his Q, it feels (even when I\'m playing him) like there\'s some skill involved. And, the enemy has some indication and time to react. Felt good.</p>\n\n<p>He feels way slower than normal and it\'s definitely noticed, especially on an already slow champ. Could be that Trinity Force is a rush item on him (it\'s frequently built live anyways) for the Movespeed alone. Without I was having a heckuva time catching up to targets. 

In [79]:
from whoosh.scoring import WeightingModel, WeightLengthScorer
from whoosh.compat import iteritems


def Custombm25(idf, tf, fl, avgfl, B, K1):
    # idf - inverse document frequency
    # tf - term frequency in the current document
    # fl - field length in the current document
    # avgfl - average field length across documents in collection
    # B, K1 - free paramters

    return idf * ((tf * (K1 + 1)) / (tf + K1 * ((1 - B) + B * fl / avgfl)))


class CustomBM25F(WeightingModel):
    """Implements the BM25F scoring algorithm.
    """

    def __init__(self, B=0.75, K1=1.2, **kwargs):
        """

        >>> from whoosh import scoring
        >>> # Set a custom B value for the "content" field
        >>> w = scoring.BM25F(B=0.75, content_B=1.0, K1=1.5)

        :param B: free parameter, see the BM25 literature. Keyword arguments of
            the form ``fieldname_B`` (for example, ``body_B``) set field-
            specific values for B.
        :param K1: free parameter, see the BM25 literature.
        """

        self.B = B
        self.K1 = K1

        self._field_B = {}
        for k, v in iteritems(kwargs):
            if k.endswith("_B"):
                fieldname = k[:-2]
                self._field_B[fieldname] = v
        

    def supports_block_quality(self):
        return True

    def scorer(self, searcher, fieldname, text, qf=1):
        if not searcher.schema[fieldname].scorable:
            return WeightScorer.for_(searcher, fieldname, text)

        if fieldname in self._field_B:
            B = self._field_B[fieldname]
        else:
            B = self.B
        print(B)
        return CustomBM25FScorer(searcher, fieldname, text, B, self.K1, qf=qf)


class CustomBM25FScorer(WeightLengthScorer):
    def __init__(self, searcher, fieldname, text, B, K1, qf=1):
        # IDF and average field length are global statistics, so get them from
        # the top-level searcher
        parent = searcher.get_parent()  # Returns self if no parent
        self.idf = parent.idf(fieldname, text)
        self.avgfl = parent.avg_field_length(fieldname) or 1

        self.B = B
        self.K1 = K1
        self.qf = qf
        self.setup(searcher, fieldname, text)

    def _score(self, weight, length):
        s = Custombm25(self.idf, weight, length, self.avgfl, self.B, self.K1)
        print(s, self.idf, weight, length, self.avgfl)
        return s

In [80]:
parser = QueryParser("post_content", ix.schema)
parser.add_plugin(FuzzyTermPlugin())
myquery = parser.parse("Juggernauts")
with ix.searcher(weighting=CustomBM25F) as searcher:
    results = searcher.search(myquery, limit=50)
    print(results.estimated_length())
    results_list = []
    for i in results:
        results_list.append((dict(i), i.score))
        
results_list

0.75
15.345098765800898 7.490739592052767 7.0 12 49.646102462405565
15.345098765800898 7.490739592052767 7.0 12 49.646102462405565
8.268893284262415 7.490739592052767 2.0 93 49.646102462405565
2.7995806278489592 7.490739592052767 1.0 253 49.646102462405565
2.8793884986754965 7.490739592052767 1.0 244 49.646102462405565
9.399699949024708 7.490739592052767 1.0 25 49.646102462405565
6.196225913167645 7.490739592052767 1.0 75 49.646102462405565
2.8793884986754965 7.490739592052767 1.0 244 49.646102462405565
3.684694207382314 7.490739592052767 1.0 175 49.646102462405565
4.121771675090212 7.490739592052767 4.0 645 49.646102462405565
2.102168379668632 7.490739592052767 2.0 738 49.646102462405565
7.062756625154789 7.490739592052767 1.0 57 49.646102462405565
1.9312373351860699 7.490739592052767 1.0 399 49.646102462405565
1.193437779551956 7.490739592052767 1.0 690 49.646102462405565
6.748182996080228 7.490739592052767 1.0 63 49.646102462405565
1.8715957832599128 7.490739592052767 1.0 414 49.646

[({'author': 'Statikk',
   'category': 'Champions & Gameplay Feedback - Now Disabled',
   'date': datetime.datetime(2015, 8, 5, 2, 50, 7, 69000, tzinfo=tzlocal()),
   'link': 'https://boards.pbe.leagueoflegends.com/en/c/champions-gameplay-feedback/Grx91k8I-darius-pbe-changelist-and-feedback?comment=000b00020000',
   'parent_content': '<p>So, Statikk, played my first PBE match with Darius and have to say he felt in such a brief testing way more fair. Such a powerful champ that was able to frontload so much damage in lane always felt super hokey. Now with the delay on his Q, it feels (even when I\'m playing him) like there\'s some skill involved. And, the enemy has some indication and time to react. Felt good.</p>\n\n<p>He feels way slower than normal and it\'s definitely noticed, especially on an already slow champ. Could be that Trinity Force is a rush item on him (it\'s frequently built live anyways) for the Movespeed alone. Without I was having a heckuva time catching up to targets. 